In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install psycopg2-binary


In [ ]:
import pandas as pd
import requests
import glob

import psycopg2
from sqlalchemy import create_engine

host="localhost"
database="censusdata"
user="user"
password="password"
port="5432"

connection = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port,
)
connection.autocommit = True
cursor = connection.cursor()
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [ ]:
# Create censusdata table in database
# Structure of table
# --------------------------------------
# + geographycode + key + value + type +
# --------------------------------------

# geographycode is the 'name' of the OA
# key is the specific census question respsonse, e.g. KS102EW0012 corresponds to the number of individuals Aged 45 to 59 for that OA
# value holsd the actual value for this census answer
# type holds the first part of the key as this corresponds to a particular census question

In [ ]:
def DownloadFile(url, local_filename):
    r = requests.get(url)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:  # filter out keep-alive new chunks
                f.write(chunk)
    return

In [ ]:
import time

def loadCensusData(censusUrlCode, censusCode):
    print(f'------- Downloading data for {censusCode} ------------')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265926TYPE299',
        f'{censusCode}-East.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265924TYPE299',
        f'{censusCode}-East-Midlands.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265927TYPE299',
        f'{censusCode}-London.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265921TYPE299',
        f'{censusCode}-North-East.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265922TYPE299',
        f'{censusCode}-North-West.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265928TYPE299',
        f'{censusCode}-South-East.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265929TYPE299',
        f'{censusCode}-South-West.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265930TYPE299',
        f'{censusCode}-Wales.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265925TYPE299',
        f'{censusCode}-West-Midlands.csv')
    DownloadFile(
        f'https://www.nomisweb.co.uk/api/v01/dataset/{censusUrlCode}.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265923TYPE299',
        f'{censusCode}-Yorkshire.csv')

    df = pd.DataFrame()
    df = pd.concat(map(pd.read_csv, glob.glob(f'./{censusCode}*.csv')))

    columnHeadings = pd.read_sql(
        f"select * from censusmetadata where code like '{censusCode}%%' and (unit = 'Count' or unit='Sum' or unit='Years')", engine)

    # Drop columns date, geography code, Rural Urban
    df.drop(['date', 'geography code', 'Rural Urban'], axis=1, inplace=True)
    if 'Religion: Has religion; measures: Value' in df.columns:
        df.drop(['Religion: Has religion; measures: Value'], axis=1, inplace=True)
    if 'Tenure: Owned; measures: Value' in df.columns:
        df.drop(['Tenure: Owned; measures: Value'], axis=1, inplace=True)
    if 'Tenure: Private rented; measures: Value' in df.columns:
        df.drop(['Tenure: Private rented; measures: Value'], axis=1, inplace=True)
    if 'Tenure: Social rented; measures: Value' in df.columns:
        df.drop(['Tenure: Social rented; measures: Value'], axis=1, inplace=True)
    if 'Country of Birth: United Kingdom; measures: Value' in df.columns:
        df.drop(['Country of Birth: United Kingdom; measures: Value'], axis=1, inplace=True)
    if 'Country of Birth: Other EU; measures: Value' in df.columns:
        df.drop(['Country of Birth: Other EU; measures: Value'], axis=1, inplace=True)
    #201
    if 'Ethnic Group: White; measures: Value' in df.columns:
        df.drop(['Ethnic Group: White; measures: Value'], axis=1, inplace=True)
    if 'Ethnic Group: Mixed/multiple ethnic groups; measures: Value' in df.columns:
        df.drop(['Ethnic Group: Mixed/multiple ethnic groups; measures: Value'], axis=1, inplace=True)
    if 'Ethnic Group: Asian/Asian British; measures: Value' in df.columns:
        df.drop(['Ethnic Group: Asian/Asian British; measures: Value'], axis=1, inplace=True)
    if 'Ethnic Group: Black/African/Caribbean/Black British; measures: Value' in df.columns:
        df.drop(['Ethnic Group: Black/African/Caribbean/Black British; measures: Value'], axis=1, inplace=True)
    if 'Ethnic Group: Other ethnic group; measures: Value' in df.columns:
        df.drop(['Ethnic Group: Other ethnic group; measures: Value'], axis=1, inplace=True)

    #105

    if 'Household Composition: One person household; measures: Value' in df.columns:
        df.drop(['Household Composition: One person household; measures: Value'], axis=1, inplace=True)
    if 'Household Composition: One family household; measures: Value' in df.columns:
        df.drop(['Household Composition: One family household; measures: Value'], axis=1, inplace=True)
    if 'Household Composition: One family only: Married or same-sex civil partnership couple; measures: Value' in df.columns:
        df.drop(['Household Composition: One family only: Married or same-sex civil partnership couple; measures: Value'], axis=1, inplace=True)
    if 'Household Composition: One family only: Cohabiting couple; measures: Value' in df.columns:
        df.drop(['Household Composition: One family only: Cohabiting couple; measures: Value'], axis=1, inplace=True)
    if 'Household Composition: One family only: Lone parent; measures: Value' in df.columns:
        df.drop(['Household Composition: One family only: Lone parent; measures: Value'], axis=1, inplace=True)
    if 'Household Composition: Other household types; measures: Value' in df.columns:
        df.drop(['Household Composition: Other household types; measures: Value'], axis=1, inplace=True)
        
    # 106
    if 'Household Composition: One person in household with a long-term health problem or disability; measures: Value' in df.columns:
        df.drop(['Household Composition: One person in household with a long-term health problem or disability; measures: Value'], axis=1, inplace=True)
    if 'Household Composition: No adults in employment in household; measures: Value' in df.columns:
        df.drop(['Household Composition: No adults in employment in household; measures: Value'], axis=1, inplace=True)
    
        
        
        


    columns = ['geographycode']
    columns = columns + (columnHeadings['code'].to_list())
    df.columns = columns

    data = []
    columns = df.columns[1:]
    for column in columns:
        df1 = pd.DataFrame()
        df1 = df[['geographycode', column]]
        df1.insert(1, "key", column)
        df1['type'] = column[:7]
        
        data = data + df1.values.tolist()

    insert_query = """insert into censusdata (geographycode, key, value, type) values %s"""
    psycopg2.extras.execute_values(
        cursor, insert_query, data, template=None, page_size=100
    )


In [ ]:

# This table provides information about the age structure, mean and median ages of usual residents of 
# England and Wales as at census day, 27 March 2011.
loadCensusData('nm_145_1','KS102EW')

In [ ]:
# classifies usual residents by their county of birth
loadCensusData('nm_611_1','KS204EW')

In [ ]:
# classifies usual residents of England and Wales by passports held
loadCensusData('nm_612_1','KS205EW')

In [ ]:
# classifies usual residents by religion
loadCensusData('nm_616_1','KS209EW')

In [ ]:
# classifies usual residents aged 16 and over by their highest level of qualification
loadCensusData('nm_623_1','KS501EW')

In [ ]:
# classifies households by tenure
loadCensusData('nm_619_1','KS402EW')

In [ ]:
# information about the number of cars or vans available to members of households
loadCensusData('nm_621_1','KS404EW')

In [ ]:
# ethnic group of the usual resident population
loadCensusData('nm_608_1','KS201EW')

In [ ]:
# information about lone parent households with dependent children
loadCensusData('nm_607_1','KS107EW')

In [ ]:
#  information that classifies households by the relationships between the household members (household composition)
loadCensusData('nm_605_1','KS105EW')

In [ ]:
# https://www.nomisweb.co.uk/api/v01/dataset/nm_625_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265926TYPE299
# Hours worked    
loadCensusData('nm_625_1','KS604EW')


In [ ]:
# https://www.nomisweb.co.uk/api/v01/dataset/nm_617_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265926TYPE299
# KS301EW - Health and provision of unpaid care
loadCensusData('nm_617_1','KS301EW')

In [ ]:
# https://www.nomisweb.co.uk/api/v01/dataset/nm_603_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265926TYPE299
# KS103EW - Marital and civil partnership status
loadCensusData('nm_603_1','KS103EW')

In [ ]:
# https://www.nomisweb.co.uk/api/v01/dataset/nm_613_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265926TYPE299
# KS206EW - Household language
loadCensusData('nm_613_1','KS206EW')

In [ ]:
# https://www.nomisweb.co.uk/api/v01/dataset/nm_606_1.bulk.csv?time=latest&measures=20100&rural_urban=total&geography=2013265926TYPE299
# KS106EW - Adults not in employment and dependent children and persons with long-term health problems or disability for all households
loadCensusData('nm_606_1','KS106EW')

In [ ]:
cursor.execute('CREATE INDEX IF NOT EXISTS idx_OA_geom ON "OA" USING gist (geometry)')
cursor.execute("CREATE INDEX idx_key on censusdata(key)")

In [ ]:
data=[]
count = cursor.execute("select count(*) from censusdata", data)
results = cursor.fetchone()
for r in results:
  print(r)
print("Total number of rows in the table:", r)
